# Ramp

Why a Ramp? 

* no spilling (jerk limit)
* less physical stress (acceleration limit, jerk limit)
* less crash demage danger (speed limit)

Watch [S-shape ramp and trapezoidal ramp](http://www.youtube.com/watch?v=qYJpl7SNoww).
Thanks to Trinamic for visualization

In [ ]:
:dep plotters = { git = "https://github.com/38/plotters", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
:dep conreg = { git = "https://github.com/almedso/conreg", default_features = false, features = ["std"] }
:sccache = 1

In [ ]:
use conreg::regulation::ramp::{Ramp, GeneralizedPosition, RampConstraints};
use std::iter;
use plotters::prelude::*;
use plotters::prelude::LineSeries;

# Plot for position ramp

Move to target position and arrive as soon as possible while maintaining speed, acceleration and jerk limits.

* trapezoidal shape ramp: (speed limit, acceleration limit)
* s-shape ramp: (speed limit, acceleration limit, jerk limit)

Examples:

* Pick and place robot
* Control a temperature profile (open jerk)

In [ ]:
let sample_interval = 0.001_f32;  // in seconds
// Instanciate and configure the ramp
let mut r = Ramp::new(sample_interval,
    RampConstraints::default()
        .max_speed(2.0)
        .max_acceleration(1.0)
        .max_jerk(1.0)
    );
r.set(GeneralizedPosition::new(5.0));
r.set_target_relative_position(-5.0);
println!("Ramp: {:?}", r);

In [ ]:
let time_range = 7.0_f32;  // in seconds
let i_range = (time_range / sample_interval ) as usize;
// Run the ramp 
let gp = r.take(i_range).collect::<Vec<GeneralizedPosition>>();

In [ ]:
// Extract position, speed and acceleration from generalized parameters for plotting
let p = gp.clone().iter().map(|x| x.position).collect::<Vec<f32>>();
let s = gp.clone().iter().map(|x| x.speed).collect::<Vec<f32>>();
let a = gp.clone().iter().map(|x| x.acceleration).collect::<Vec<f32>>();

// Prepare the plot
let figure = evcxr_figure((640, 480), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Position/speed/acceleration time diagram", ("Arial", 24).into_font())
        .margin(5)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d(0.0f32..time_range, -5.0f32..5.0f32)?;

    chart.configure_mesh().draw()?;

    chart.draw_series(LineSeries::new(
        (0..=i_range).map(|x| ( x as f32 * sample_interval)).zip(p),
        &GREEN,
    )).unwrap()
        .label("Position")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &GREEN));
    chart.draw_series(LineSeries::new(
        (0..=i_range).map(|x| ( x as f32 * sample_interval)).zip(s),
        &RED,
    )).unwrap()
        .label("Velocity")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &RED));
    chart.draw_series(LineSeries::new(
        (0..=i_range).map(|x| ( x as f32 * sample_interval)).zip(a),
        &BLUE,
    )).unwrap()
        .label("Acceleration")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &BLUE));

    chart.configure_series_labels()
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .position(SeriesLabelPosition::UpperRight)
        .draw()?;
    Ok(())
});

In [ ]:
// Show the ramp plot
figure

# Plot of speed ramp

... if you want to move at a constant speed and maintain the speed open end and softly accelerate. 

Examples:

* cruise control in cars

In [ ]:
let sample_interval = 0.001_f32;  // in seconds
// Instanciate and configure the ramp
let mut r = Ramp::new(sample_interval,
    RampConstraints::default()
        .max_speed(2.0)
        .max_acceleration(1.0)
        .max_jerk(1.0));
let gp = GeneralizedPosition {position: 0.0, speed: 2.0, acceleration: 0.0 };
r.set(gp);
r.set_target_speed(-1.0);
println!("Ramp: {:?}", r);

In [ ]:
let time_range = 5.0_f32;  // in seconds
let i_range = (time_range / sample_interval ) as usize;
// Run the ramp 
let gp = r.take(i_range).collect::<Vec<GeneralizedPosition>>();

In [ ]:
// Extract position, speed and acceleration from generalized parameters for plotting
let s = gp.clone().iter().map(|x| x.speed).collect::<Vec<f32>>();
let a = gp.clone().iter().map(|x| x.acceleration).collect::<Vec<f32>>();

// Prepare the plot
let figure = evcxr_figure((640, 480), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Position/speed/acceleration time diagram", ("Arial", 24).into_font())
        .margin(5)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d(0.0f32..time_range, -1.0f32..2.0f32)?;

    chart.configure_mesh().draw()?;

    chart.draw_series(LineSeries::new(
        (0..=i_range).map(|x| ( x as f32 * sample_interval)).zip(s),
        &RED,
    )).unwrap()
        .label("Velocity")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &RED));
    chart.draw_series(LineSeries::new(
        (0..=i_range).map(|x| ( x as f32 * sample_interval)).zip(a),
        &BLUE,
    )).unwrap()
        .label("Acceleration")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &BLUE));


    chart.configure_series_labels()
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .draw()?;
    Ok(())
});

In [ ]:
figure